## Simulation for Power Analysis

### What is Power?

- $Power = P(rejecting Null ∣ true alternative)$  
- Probability of detecting an effect if it exists. 
- Depends on sample size, α and effect size. 
- Typically 80% power recommended for α = 0.0

![DGP for Power Analysis](img/dgp_for_power.jpg)

### Power Analysis - Part I

Now we turn to power analysis. You typically want to ensure that any experiment or A/B test you run has at least 80% power. One way to ensure this is to calculate the sample size required to achieve 80% power.

Suppose that you are in charge of a news media website and you are interested in increasing the amount of time users spend on your website. Currently, the time users spend on your website is normally distributed with a mean of 1 minute and a variance of 0.5 minutes. Suppose that you are introducing a feature that loads pages faster and want to know the sample size required to measure a 10% increase in time spent on the website.

In this exercise, we will set up the framework to run one simulation, run a t-test & calculate the p-value.

In [1]:
import numpy as np
import scipy.stats as st
# Initialize effect_size, control_mean, control_sd
effect_size, sample_size, control_mean, control_sd = .05, 50, 1, .5

# Simulate control_time_spent and treatment_time_spent, assuming equal variance
control_time_spent = np.random.normal(loc=control_mean, 
                                      scale=control_sd, 
                                      size=sample_size)
treatment_time_spent = np.random.normal(loc=control_mean*(1+effect_size), 
                                        scale=control_sd, 
                                        size=sample_size)

# Run the t-test and get the p_value
t_stat, p_value = st.ttest_ind(treatment_time_spent, control_time_spent)
print("P-value: {}, Statistically Significant? {}".format(p_value, p_value < .05))

P-value: 0.838499371287144, Statistically Significant? False


### Power Analysis - Part II

Previously, we simulated one instance of the experiment & generated a p-value. We will now use this framework to calculate statistical power. Power of an experiment is the experiment's ability to detect a difference between treatment & control if the difference really exists. It's good statistical hygiene to strive for 80% power.

For our website, we want to know how many people need to visit each variant, such that we can detect a 10% increase in time spent with 80% power. For this, we start with a small sample (50), simulate multiple instances of this experiment & check power. If 80% power is reached, we stop. If not, we increase the sample size & try again.

In [2]:
sample_size = 50
sims = 1000
effect_size = .05

# Keep incrementing sample size by 10 till we reach required power
while 1:
    control_time_spent = np.random.normal(loc=control_mean, 
                                          scale=control_sd, 
                                          size=(sample_size, sims))
    treatment_time_spent = np.random.normal(loc=control_mean*(1+effect_size), 
                                            scale=control_sd, 
                                            size=(sample_size, sims))
    t, p = st.ttest_ind(treatment_time_spent, control_time_spent)
    
    # Power is the fraction of times in the simulation 
    # when the p-value was less than 0.05
    power = (p < 0.05).sum()/sims
    if power >= .8: 
        break
    else: 
        sample_size += 10
print("For 80% power, sample size required = {}".format(sample_size))

For 80% power, sample size required = 1560
